In [ ]:
import os
from dotenv import load_dotenv
from tools.VCL_query_timeseries import timeseries_query
from tools.VCL_query_timeseries_area import timeseries_area_query
from tools.VCL_get_query_status import get_query_status
from tools.VCL_get_query_result import get_query_result
from tools.VCL_query_aggregation import construct_aggregation_dict, aggregation_query
from json import JSONDecodeError
from time import sleep
import pandas as pd
from pathlib import Path

load_dotenv()
api_key = os.environ["API_SECRET_KEY"]


# Area Queries

min_longitude = 0.98<br>
max_longitude = 4.2<br>
latitudes from 51.05 until 54.8

min_longitude = -1.5<br>
max_longitude = 0.98<br>
latitudes from 51.05 until 54.8

min_longitude = 4.2<br>
max_longitude = 8.2<br>
latitudes from 51.05 until 54.8

## Make New Area Requests

In [27]:
###########
#### AREA REQUEST
##########

min_longitude = 0.98
max_longitude = 4.2

start_date = "20220101"
end_date = "20221231"
vars_4d = ["wsp","rho"]
vars_3d = ["xland"]
heights = [150]


max_step_size = 0.25
from_lat = 52.8
to_lat = 54.8
min_latitude, max_latitude = from_lat, min(to_lat, from_lat + max_step_size)

query_list = []

while True:
    response = timeseries_area_query(min_longitude, max_longitude, min_latitude, max_latitude, start_date, end_date, vars_4d, vars_3d, heights, api_key)
    try:
        response_json = response.json()
    except JSONDecodeError:
        print("error:")
        print(response.text)
    else:
        print(f"{min_latitude=}, {max_latitude=} - {response_json}")
        min_latitude = max_latitude
        max_latitude = min(to_lat, max_latitude + max_step_size)
        query_list.append({
            "query_id": response_json["queryId"],
            "min_latitude": min_latitude,
            "max_latitude": max_latitude,
        })
        
        
    
    if max_latitude >= to_lat:
        break
    sleep(60)



min_latitude=52.8, max_latitude=53.05 - {'queryId': '2f2d2044-943c-43fb-88f3-c9b8bd4d14c6'}
min_latitude=53.05, max_latitude=53.3 - {'queryId': '4917ea1b-9fa0-46f5-804c-c6486f81489d'}
min_latitude=53.3, max_latitude=53.55 - {'queryId': '4e046e09-fea1-492f-8de6-be117eaca341'}
min_latitude=53.55, max_latitude=53.8 - {'queryId': 'c6fef295-5114-4e71-bbc2-ce2db06714c4'}
min_latitude=53.8, max_latitude=54.05 - {'queryId': '283fa566-5365-4383-a1c0-bf1177878556'}
min_latitude=54.05, max_latitude=54.3 - {'queryId': 'b1eb96ba-d0a9-40d4-90ef-ef012f53029a'}
min_latitude=54.3, max_latitude=54.55 - {'queryId': '0c77605b-e3b3-4b7e-b823-bfa796b3f42d'}


In [28]:
query_list

[{'query_id': '2f2d2044-943c-43fb-88f3-c9b8bd4d14c6',
  'min_latitude': 53.05,
  'max_latitude': 53.3},
 {'query_id': '4917ea1b-9fa0-46f5-804c-c6486f81489d',
  'min_latitude': 53.3,
  'max_latitude': 53.55},
 {'query_id': '4e046e09-fea1-492f-8de6-be117eaca341',
  'min_latitude': 53.55,
  'max_latitude': 53.8},
 {'query_id': 'c6fef295-5114-4e71-bbc2-ce2db06714c4',
  'min_latitude': 53.8,
  'max_latitude': 54.05},
 {'query_id': '283fa566-5365-4383-a1c0-bf1177878556',
  'min_latitude': 54.05,
  'max_latitude': 54.3},
 {'query_id': 'b1eb96ba-d0a9-40d4-90ef-ef012f53029a',
  'min_latitude': 54.3,
  'max_latitude': 54.55},
 {'query_id': '0c77605b-e3b3-4b7e-b823-bfa796b3f42d',
  'min_latitude': 54.55,
  'max_latitude': 54.8}]

## Check Query Status and download completed

In [29]:
def check_for_completed_query(query_info: dict, query_list: list, completed_queries: list) -> bool:
    response = get_query_status(query_info["query_id"], api_key)
    try:
        response_json = response.json()
    except Exception:
        print(response.text)
        query_list.append(query_info)
    else:
        if response_json.get("currentState", None) == "Finished":
            completed_queries.append(query_info)
            print(f"Finished: {query_info}")
            return True
        else:
            query_list.append(query_info)
            if response_json.get("currentState") is not None:
                print(f"{response_json.get('currentState')}: {query_info}")
                return False
            else:
                print(f"{response_json}: {query_info}")
                return False

completed_queries = []
downloaded_queries = []
while len(query_list) > 0 or len(completed_queries) > 0:
    
    if len(completed_queries) == 0:
        query_info = query_list.pop(0)
        print("Checking for completion:")
        if not check_for_completed_query(query_info, query_list, completed_queries):
            sleep(60)
    else:
        query_info = completed_queries.pop(0)
        print(f"staring download for {query_info['query_id']}")
        stuff = get_query_result(query_info["query_id"], api_key, 10000000, 1)
        try:
            stuff = stuff.json()
        except Exception:
            print(f"Failed download {query_info}")
            completed_queries.append(query_info)
        if len(stuff) == 1:
            print(f"failed download: {query_info}")
            completed_queries.append(query_info)
            sleep(60)
        else:
            df = pd.DataFrame.from_records(stuff, index=range(len(stuff)))
            df.to_csv(f"area_wind_speed_data_bigger{query_info['query_id'][:4]}.csv", mode="a", index=False)
            downloaded_queries.append(query_info)
            print(f"downloaded ({len(stuff)}) area_wind_speed_data_bigger{[query_info['query_id'][:4]]}.csv")
            

Checking for completion:
Running: {'query_id': '2f2d2044-943c-43fb-88f3-c9b8bd4d14c6', 'min_latitude': 53.05, 'max_latitude': 53.3}
Checking for completion:
Finished: {'query_id': '4917ea1b-9fa0-46f5-804c-c6486f81489d', 'min_latitude': 53.3, 'max_latitude': 53.55}
staring download for 4917ea1b-9fa0-46f5-804c-c6486f81489d
downloaded (5641440) area_wind_speed_data_bigger['4917'].csv
Checking for completion:
Running: {'query_id': '4e046e09-fea1-492f-8de6-be117eaca341', 'min_latitude': 53.55, 'max_latitude': 53.8}
Checking for completion:
Running: {'query_id': 'c6fef295-5114-4e71-bbc2-ce2db06714c4', 'min_latitude': 53.8, 'max_latitude': 54.05}
Checking for completion:
Running: {'query_id': '283fa566-5365-4383-a1c0-bf1177878556', 'min_latitude': 54.05, 'max_latitude': 54.3}
Checking for completion:
Finished: {'query_id': 'b1eb96ba-d0a9-40d4-90ef-ef012f53029a', 'min_latitude': 54.3, 'max_latitude': 54.55}
staring download for b1eb96ba-d0a9-40d4-90ef-ef012f53029a
downloaded (5510040) area_win

## Combine Downloaded CSVs and Compress

In [ ]:
paths = [path for path in Path().glob("area_wind_speed_data*.csv")]
frames = [pd.read_csv(path) for path in paths]

df = pd.concat(frames)

filename = 'area_wind_speed.csv'
compression_options = dict(method='zip', archive_name=f'{filename}')
df.to_csv(f"input_data/timeseries_area_wind_speed/{filename}.zip", compression=compression_options, index=False)

# Other Query Types (Single Point & Aggregation)

In [5]:
### AGGREGATION QUERY
min_longitude = -4.4454
max_longitude = 12.0059
min_latitude = 53.9954
max_latitude = 56.9954
start_date = "20120101"
end_date = "20121231"
agg_functions = ["AVG", "MAX", "MIN", "SD"] # At least one function is required

vars_3d = ["xland"]
vars_4d = ["wsp"]
heights = [150] 
aggregation = construct_aggregation_dict(agg_functions, vars_3d, vars_4d, heights)

response = aggregation_query(min_longitude, max_longitude, min_latitude, max_latitude, start_date, end_date, aggregation, api_key)
print("Aggregation Query: ")
try:
    response_json = response.json()
    print(response_json)
except JSONDecodeError:
    print("error:", end="\t")
    print(response.text)

Aggregation Query: 
Timeseries Area Query: 
{'queryId': '1ad06c41-c651-4919-8750-7ab159018207'}


In [ ]:
# 2022
# min_latitude = 50.9954, max_latitude = 53.9954
# {'queryId': '6a117fc4-030a-4584-8a3b-485d28f520ee'}
# min_latitude = 53.9954, max_latitude = 56.9954
# {'queryId': 'fee7d638-2254-412f-9cd0-60a9576c184a'}

# 2012
# min_latitude = 50.9954, max_latitude = 53.9954
# {'queryId': '0fcc4e59-bf75-4cd0-a52c-2ed46ddad3a0'}
# min_latitude = 53.9954, max_latitude = 56.9954
# {'queryId': '1ad06c41-c651-4919-8750-7ab159018207'}

In [23]:
###########
#### POINT REQUEST
##########

latitude = 53.62
longitude = 4.42
start_date = "2012-01-01"
end_date = "2022-12-31"
vars_4d = ["wsp","wdir", "rho"] # Can be empty []
vars_3d = ["xland"]
heights = [150] # Must not be empty if vars_4d is not empty


response = timeseries_query(latitude, longitude, start_date, end_date, vars_4d, vars_3d, heights, api_key)
try:
    response_json = response.json()
    query_id = response_json["queryId"]
    print("Point Query: ")
    print(response_json)
except JSONDecodeError:
    print("error:")
    print(response.text)

Point Query: 
{'queryId': 'b90b21cd-2035-4148-a359-99a867988a3f'}
